# Multithreading in Julia

_Part of this notebook is inspired by the material of th [Julia for HPC Course @ UCL ARC ](https://github.com/carstenbauer/JuliaUCL24) by Carsten Bauer._

## Setup

In [1]:
# Running this cell is important to make sure we install all the necessary packages.
using Pkg
Pkg.instantiate()

## Spawning parallel tasks

In [2]:
using Base.Threads

@show nthreads();

nthreads() = 4


In [3]:
@time t = @spawn begin # `@spawn` returns right away
    sleep(2)
    3+3
end

@time fetch(t) # `fetch` waits for the task to finish

  0.006308 seconds (2.80 k allocations: 202.039 KiB, 99.16% compilation time)
  1.974778 seconds (276 allocations: 17.109 KiB, 0.18% compilation time)


6

## Example: multi-threaded `map`

In [4]:
using LinearAlgebra, BenchmarkTools

BLAS.set_num_threads(1) # Fix number of BLAS threads

function tmap(fn, itr)
    # for each i ∈ itr, spawn a task to compute fn(i)
    tasks = map(i -> @spawn(fn(i)), itr)
    # fetch and return all the results
    return fetch.(tasks)
end

M = [rand(200,200) for i in 1:8];

tmap(svdvals, M);

@btime  map(svdvals, $M) samples=10 evals=3;
@btime tmap(svdvals, $M) samples=10 evals=3;

  21.623 ms (65 allocations: 3.37 MiB)
  6.112 ms (113 allocations: 3.38 MiB)


***Exercise***: do you see any difference if you increase the number of BLAS threads?

## Example: multi-threaded `for` loop (reduction)

In [5]:
using ChunkSplitters, Base.Threads, BenchmarkTools

function sum_threads(fn, data; nchunks=nthreads())
    psums = zeros(eltype(data), nchunks)
    @threads :dynamic for (c, elements) in enumerate(chunks(data; n=nchunks))
        psums[c] = sum(fn, elements)
    end
    return sum(psums)
end

v = randn(10_000_000);

@btime sum(sin, $v);

@btime sum_threads(sin, $v);

  118.874 ms (0 allocations: 0 bytes)
  29.698 ms (25 allocations: 2.36 KiB)


***Exercise***: do you see differences if you change the scheduler type?  Remember you can choose between `:dynamic` (currently the default if omitted), `:greedy` (only if using Julia v1.11+), and `:static`.

In [6]:
function sum_map_spawn(fn, data; nchunks=nthreads())
    ts = map(chunks(data, n=nchunks)) do elements
        @spawn sum(fn, elements)
    end
    return sum(fetch.(ts))
end

@btime sum_map_spawn(sin, $v);

  29.746 ms (40 allocations: 2.78 KiB)


In [7]:
using OhMyThreads: @tasks

function sum_tasks(fn, data; nchunks=nthreads())
    psums = zeros(eltype(data), nchunks)
    @tasks for (c, elements) in enumerate(chunks(data; n=nchunks))
        psums[c] = sum(fn, elements)
    end
    return sum(psums)
end

@btime sum_tasks(sin, $v);

  29.927 ms (32 allocations: 2.66 KiB)


In [8]:
using OhMyThreads: tmapreduce

@btime tmapreduce(sin, +, $v);

  30.202 ms (93 allocations: 7.42 KiB)


## Multi-threading: is it always worth it?

In [9]:
using BenchmarkTools

function overhead!(v)
    for idx in eachindex(v)
        v[idx] = idx
    end
end
    
function overhead_threads!(v)
    @threads for idx in eachindex(v)
        v[idx] = idx
    end
end

N = 100

@btime overhead!(v) setup=(v = Vector{Int}(undef, N))
@btime overhead_threads!(v) setup=(v = Vector{Int}(undef, N))

  16.504 ns (0 allocations: 0 bytes)
  5.402 μs (21 allocations: 2.08 KiB)


***Exercise***: do you see any improvement in the parallel efficiency if you change the size of the problem (here: `N`)?

## Unbalanced workload: computing hexadecimal $\pi$

_This section is inspired by the blogpost [Computing the hexadecimal value of pi](https://giordano.github.io/blog/2017-11-21-hexadecimal-pi/) by Mosè Giordano._

The [Bailey–Borwein–Plouffe formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula) is one of the [several algorithms to compute $\pi$](https://en.wikipedia.org/wiki/Approximations_of_%CF%80):

$$
\pi = \sum_{k = 0}^{\infty}\left[ \frac{1}{16^k} \left( \frac{4}{8k + 1} -
\frac{2}{8k + 4} - \frac{1}{8k + 5} - \frac{1}{8k + 6} \right) \right]
$$

What makes this formula stand out among other approximations of $\pi$ is that it allows one to directly extract the $n$-th fractional digit of the hexadecimal value of $\pi$ without computing the preceding ones.

The Wikipedia article about the Bailey–Borwein–Plouffe formula explains that the $n + 1$-th fractional digit $d_n$ is given by

$$
d_{n} = 16 \left[ 4 \Sigma(n, 1) - 2 \Sigma(n, 4) - \Sigma(n, 5) - \Sigma(n,
6) \right]
$$

where

$$
\Sigma(n, j) = \sum_{k = 0}^{n} \frac{16^{n-k} \bmod (8k+j)}{8k+j} + \sum_{k
= n+1}^{\infty} \frac{16^{n-k}}{8k+j}
$$

Only the fractional part of expression in square brackets on the right side of $d_n$ is relevant, thus, in order to avoid rounding errors, when we compute each term of the finite sum above we can take only the fractional part. This allows us to always use ordinary double precision floating-point arithmetic, without resorting to arbitrary-precision numbers. In addition note that the terms of the infinite sum get quickly very small, so we can stop the summation when they become negligible.

### Serial implementation

In [10]:
# Return the fractional part of x, modulo 1, always positive
fpart(x) = mod(x, one(x))

function Σ(n, j)
    # Compute the finite sum
    s = 0.0
    denom = j
    for k in 0:n
        s = fpart(s + powermod(16, n - k, denom) / denom)
        denom += 8
    end
    # Compute the infinite sum
    num = 1 / 16
    while (frac = num / denom) > eps(s)
        s     += frac
        num   /= 16
        denom += 8
    end
    return fpart(s)
end

pi_digit(n) =
    floor(Int, 16 * fpart(4Σ(n-1, 1) - 2Σ(n-1, 4) - Σ(n-1, 5) - Σ(n-1, 6)))

pi_string(n) = "0x3." * join(string.(pi_digit.(1:n), base = 16)) * "p0"

pi_string (generic function with 1 method)

Let's make sure this works:

In [11]:
pi_string(13)

"0x3.243f6a8885a30p0"

In [12]:
# Parse the string as a double-precision floating point number
parse(Float64, pi_string(13))

3.141592653589793

In [13]:
Float64(π) == parse(Float64, pi_string(13))

true

In [14]:
setprecision(BigFloat, 4000) do
    BigFloat(π) == parse(BigFloat, pi_string(1000))
end

true

In [15]:
using BenchmarkTools

b = @benchmark pi_string(1000)

pi_serial_t = minimum(b.times)

b

BenchmarkTools.Trial: 21 samples with 1 evaluation.
 Range (min … max):  248.039 ms … 260.443 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     248.999 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   249.864 ms ±   2.713 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ █ █▄                                                         
  █▁█▆██▆▆▁▆▁▆▁▁▁▁▁▁▁▆▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  248 ms           Histogram: frequency by time          260 ms <

 Memory estimate: 98.05 KiB, allocs estimate: 2009.

### Multi-threaded implementation

Since the Bailey–Borwtimesn–Plouffe formula extracts the $n$-th digit of $\pi$ without computing the other ones, we can write a multi-threaded version of `pi_string`, taking advantage of native support for [multi-threading](https://docs.julialang.org/en/v1/manual/multi-threading/) in Julia. However note that the computational cost of `pi_digit` is $O(n\log(n))$, so the larger the value of $n$, the longer the function will take, which makes this workload very unbalanced. ***Question***: what do you expect to be the worst performing scheduler?

#### For-loop: static scheduler

In [16]:
function pi_string_threads_static(N)
    digits = Vector{Int}(undef, N)
    @threads :static for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_threads_static(1000) == pi_string(1000)

true

In [17]:
b = @benchmark pi_string_threads_static(1000)

pi_threads_static_t = minimum(b.times)

b

BenchmarkTools.Trial: 43 samples with 1 evaluation.
 Range (min … max):  115.481 ms … 125.829 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     116.675 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   117.656 ms ±   2.632 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅  ▅ ▂█▅    ▂                                                  
  ████▅███▅▅█▅█▁█▁▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁█▁▁▅▁▁▁▁▁▁▁▁▅▁▁▁▅ ▁
  115 ms           Histogram: frequency by time          126 ms <

 Memory estimate: 108.09 KiB, allocs estimate: 2033.

In [18]:
pi_serial_t / pi_threads_static_t / nthreads() * 100

53.696901830786146

#### For-loop: dynamic scheduler

In [19]:
function pi_string_threads_dynamic(N)
    digits = Vector{Int}(undef, N)
    @threads :dynamic for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_threads_dynamic(1000) == pi_string(1000)

true

In [20]:
b = @benchmark pi_string_threads_dynamic(1000)

pi_threads_dynamic_t = minimum(b.times)

b

BenchmarkTools.Trial: 43 samples with 1 evaluation.
 Range (min … max):  115.459 ms … 129.002 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     116.613 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   117.295 ms ±   2.522 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂▂ █                                                         
  ▅▅████▅▇█▄▁▁▁▁▅▁▁▁▄▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  115 ms           Histogram: frequency by time          129 ms <

 Memory estimate: 108.09 KiB, allocs estimate: 2033.

In [21]:
pi_serial_t / pi_threads_dynamic_t / nthreads() * 100

53.70715342253043

#### For-loop: greedy scheduler (only Julia v1.11+)

In [22]:
@static if VERSION >= v"1.11"

function pi_string_threads_greedy(N)
    digits = Vector{Int}(undef, N)
    @threads :greedy for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_threads_greedy(1000) == pi_string(1000)

end

In [23]:
@static if VERSION >= v"1.11"

b = @benchmark pi_string_threads_greedy(1000)

pi_threads_greedy_t = minimum(b.times)

b

end

In [24]:
@static if VERSION >= v"1.11"

pi_serial_t / pi_threads_greedy_t / nthreads() * 100

end

#### Tasks

In [25]:
function pi_string_tasks(N)
    tasks = [Threads.@spawn pi_digit(n) for n in 1:N]
    digits = [fetch(t) for t in tasks]
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_tasks(1000) == pi_string(1000)

true

In [26]:
b = @benchmark pi_string_tasks(1000)

pi_tasks_t = minimum(b.times)

b

BenchmarkTools.Trial: 80 samples with 1 evaluation.
 Range (min … max):  62.264 ms … 69.605 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     62.776 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   63.120 ms ±  1.070 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    █ ▆ ▄                                                      
  ▄▁█████▃▇▆▁▄▄▄▃▄▁▃▃▅▄▄▁▁▁▁▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  62.3 ms         Histogram: frequency by time        67.1 ms <

 Memory estimate: 598.48 KiB, allocs estimate: 7017.

In [27]:
pi_serial_t / pi_tasks_t / nthreads() * 100

99.59224985224948

#### OhMyThreads.jl

In [28]:
using OhMyThreads: @tasks

function pi_string_omt(N; ntasks::Int=8 * nthreads(), scheduler::Symbol=:dynamic)
    digits = Vector{Int}(undef, N)
    @tasks for n in eachindex(digits)
        @set ntasks=ntasks
        @set scheduler=scheduler
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_omt(1000) == pi_string(1000)

true

In [29]:
b = @benchmark pi_string_omt(1000)

pi_omt_t = minimum(b.times)

b

BenchmarkTools.Trial: 76 samples with 1 evaluation.
 Range (min … max):  63.664 ms … 73.122 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     65.719 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   66.174 ms ±  1.743 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▂█▂ ▄  ▄    ▂▂                                        
  ▄▄▄▄▆▁██████▄▄█▆█▄▆██▆▆▁▄▆▄▆▄▆▁▁▄▁▄▄▁▁▁▁▁▆▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  63.7 ms         Histogram: frequency by time        72.1 ms <

 Memory estimate: 123.56 KiB, allocs estimate: 2213.

In [30]:
pi_serial_t / pi_omt_t / nthreads() * 100

97.40242422153793